In [3]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import dask.dataframe as dd
import dask.distributed
import pyarrow.parquet as pq




## Conn

In [1]:
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError
import os
import pandas as pd
from io import BytesIO
from dotenv import load_dotenv
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Conectarse al servicio de Blob Storage
account_name = os.getenv("ACCOUNT_NAME")
account_key = os.getenv("ACCOUNT_KEY")
container_name = 'getdatalakefiles'

blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

## Yellow

In [2]:
def list_parquet_files(blob_service_client, container_name, path):
    """
    Lista los archivos parquet en la ruta especificada dentro de un contenedor.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - path (str): La ruta dentro del contenedor.

    Returns:
    - list: Lista de nombres de archivos parquet en la ruta especificada.
    """
    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)
    
    # Listar los blobs en la ruta especificada con extensión ".parquet"
    blobs = container_client.list_blobs(name_starts_with=path, include=['metadata'])
    
    # Filtrar los blobs con extensión ".parquet" y obtener sus nombres
    parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]
    
    return parquet_files

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'path' con la ruta deseada dentro del contenedor
parquet_files = list_parquet_files(blob_service_client, container_name, "raw-data/yellow_analytics/")


In [3]:
parquet_files

['raw-data/yellow_analytics/yellow_analytics_part_10_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_3.parquet',
 'raw-data/yellow_analyti

In [4]:
# Obtener el cliente del contenedor y el blob
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client("raw-data/yellow_analytics/yellow_analytics_part_1_1.parquet")

# Descargar el archivo desde Azure Blob Storage
downloaded_blob = blob_client.download_blob()

# Leer el archivo parquet en un DataFrame de pandas
with BytesIO() as file_stream:
    downloaded_blob.readinto(file_stream)
    file_stream.seek(0)
    df = pd.read_parquet(file_stream)


In [5]:
df.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
517030,1,1,2.7,1.0,264,264,1,19.56,2022-01-08,14,1355
397151,1,1,1.5,1.0,236,238,1,14.16,2022-01-06,19,417
435109,1,1,2.0,1.0,236,142,1,13.00,2022-01-07,13,460
10724,1,1,4.0,1.0,164,263,1,23.15,2022-01-01,2,1032
612952,1,1,2.7,1.0,236,48,1,15.87,2022-01-09,18,678


In [6]:
df.shape

(272563, 11)

In [7]:
def read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files):
    """
    Lee los archivos parquet y los une en un DataFrame.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - parquet_files (list): Lista de nombres de archivos parquet en el contenedor.

    Returns:
    - pd.DataFrame: DataFrame que contiene todos los datos de los archivos parquet.
    """
    dfs = []

    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)

    for file_name in parquet_files:
        # Obtener el cliente del blob
        blob_client = container_client.get_blob_client(file_name)

        # Descargar el archivo desde Azure Blob Storage
        downloaded_blob = blob_client.download_blob()

        # Leer el archivo parquet en un DataFrame de pandas
        with BytesIO() as file_stream:
            downloaded_blob.readinto(file_stream)
            file_stream.seek(0)
            df = pd.read_parquet(file_stream)
            dfs.append(df)

    # Concatenar todos los DataFrames en uno solo
    result_df = pd.concat(dfs, ignore_index=True)
    return result_df

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'parquet_files' con la lista de nombres de archivos parquet
df = read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files)

In [8]:
df.shape

(38287508, 11)

In [19]:
df.dtypes

IdProveedor         int64
TotalPasajeros      int32
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int32
DuracionViaje       int32
dtype: object

In [11]:
df.head(5)

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
0,1,1,1.70,1.0,249,107,1,15.95,2022-10-01,0,898
1,2,1,1.74,1.0,238,166,1,12.36,2022-10-01,0,628
2,1,1,6.80,1.0,186,41,2,29.30,2022-10-01,0,1762
3,2,3,1.88,1.0,162,145,2,14.30,2022-10-01,0,692
4,2,1,7.79,1.0,138,112,1,31.01,2022-10-01,0,1083


In [20]:
# Convertir la columna 'FechaRecogida' a tipo datetime si aún no lo está
df['FechaRecogida'] = pd.to_datetime(df['FechaRecogida'])

In [21]:
# Para crear una columna solo con el año
df['Anio'] = df['FechaRecogida'].dt.year

In [22]:
df.dtypes

IdProveedor                int64
TotalPasajeros             int32
DistanciaViaje           float64
IdTipoTarifa             float64
IdZonaOrigen               int64
IdZonaDestino              int64
IdTipoPago                 int64
CostoTotal               float64
FechaRecogida     datetime64[ns]
HoraRecogida               int32
DuracionViaje              int32
Anio                       int32
dtype: object

In [23]:
df.Anio.unique()

array([2022, 2023])

## FFHV

In [2]:
def list_parquet_files(blob_service_client, container_name, path):
    """
    Lista los archivos parquet en la ruta especificada dentro de un contenedor.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - path (str): La ruta dentro del contenedor.

    Returns:
    - list: Lista de nombres de archivos parquet en la ruta especificada.
    """
    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)
    
    # Listar los blobs en la ruta especificada con extensión ".parquet"
    blobs = container_client.list_blobs(name_starts_with=path, include=['metadata'])
    
    # Filtrar los blobs con extensión ".parquet" y obtener sus nombres
    parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]
    
    return parquet_files

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'path' con la ruta deseada dentro del contenedor
parquet_files = list_parquet_files(blob_service_client, container_name, "raw-data/ffvh_analytics/")

In [3]:
parquet_files

['raw-data/ffvh_analytics/ffvh_analytics_part_0_1.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_0_2.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_0_3.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_0_4.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_10_1.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_10_2.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_10_3.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_10_4.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_11_1.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_11_2.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_11_3.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_11_4.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_12_1.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_12_2.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_12_3.parquet',
 'raw-data/ffvh_analytics/ffvh_analytics_part_12_4.parquet',
 'raw-data/ffvh_analytics/ff

In [4]:
def read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files, max_retries=3, retry_delay=5):
    """
    Lee los archivos parquet y los une en un DataFrame.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - parquet_files (list): Lista de nombres de archivos parquet en el contenedor.
    - max_retries (int): Número máximo de intentos de descarga.
    - retry_delay (int): Tiempo de espera entre reintentos en segundos.

    Returns:
    - pd.DataFrame: DataFrame que contiene todos los datos de los archivos parquet.
    """
    dfs = []

    container_client = blob_service_client.get_container_client(container_name)

    for file_name in parquet_files:
        retry_count = 0
        success = False

        while retry_count < max_retries and not success:
            try:
                blob_client = container_client.get_blob_client(file_name)
                downloaded_blob = blob_client.download_blob()

                with BytesIO() as file_stream:
                    downloaded_blob.readinto(file_stream)
                    file_stream.seek(0)
                    df = pd.read_parquet(file_stream)
                    dfs.append(df)

                success = True
            except ResourceNotFoundError as e:
                print(f"Error: El archivo {file_name} no se encontró.")
                break
            except Exception as e:
                print(f"Error al descargar {file_name}: {str(e)}")
                retry_count += 1
                if retry_count < max_retries:
                    print(f"Reintentando en {retry_delay} segundos...")
                    time.sleep(retry_delay)
                else:
                    print(f"Se alcanzó el número máximo de reintentos para {file_name}.")
    
    result_df = pd.concat(dfs, ignore_index=True)
    return result_df

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'path' con la ruta deseada dentro del contenedor
df = read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files)

In [5]:
df.shape

(26265587, 17)

In [6]:
df.head(5)

,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,255,33,3.60,965,18.74,34.14,2022-01-05,10:40,2022-01-05,10:56,2022-01-05,10:37,2022-01-05,10:38,71.0
1,HV0003,B03404,21,149,1.96,464,11.03,19.31,2022-01-07,20:18,2022-01-07,20:26,2022-01-07,20:14,2022-01-07,20:18,220.0
2,HV0003,B03404,65,177,4.64,1246,24.01,43.86,2022-01-08,01:12,2022-01-08,01:33,2022-01-08,01:05,2022-01-08,01:12,381.0
3,HV0003,B03404,39,155,2.05,633,8.96,17.71,2022-01-04,17:49,2022-01-04,18:00,2022-01-04,17:45,2022-01-04,17:47,152.0
4,HV0003,B03404,35,77,1.65,475,7.51,15.57,2022-01-06,02:22,2022-01-06,02:30,2022-01-06,02:17,2022-01-06,02:22,258.0


In [26]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/yellow_analytics/yellow_analytics_part_2_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(487229, 11)


,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
0,1,1,5.40,1.0,138,252,1,23.45,2022-02-01,0,746
1,1,1,6.40,1.0,138,41,2,30.10,2022-02-01,0,1053
2,1,1,12.50,1.0,138,200,2,44.60,2022-02-01,0,1419
3,2,1,9.88,1.0,239,200,2,34.80,2022-02-01,0,1205
4,2,1,12.16,1.0,138,125,1,48.66,2022-02-01,0,1579


In [ ]:
print(df.head(5))

In [9]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/ffvh_analytics/ffvh_analytics_part_0_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(2303347, 17)


,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,170,161,1.18,664,24.90,53.64,2022-01-01,00:07,2022-01-01,00:18,2022-01-01,00:05,2022-01-01,00:05,9.0
1,HV0003,B03404,237,161,0.82,460,11.97,28.46,2022-01-01,00:22,2022-01-01,00:30,2022-01-01,00:19,2022-01-01,00:22,161.0
3,HV0003,B03404,262,229,1.65,303,7.91,17.90,2022-01-01,00:18,2022-01-01,00:23,2022-01-01,00:15,2022-01-01,00:17,92.0
4,HV0003,B03404,229,141,1.65,461,9.44,20.75,2022-01-01,00:28,2022-01-01,00:35,2022-01-01,00:25,2022-01-01,00:26,16.0
5,HV0003,B03404,263,79,4.51,762,17.67,34.77,2022-01-01,00:38,2022-01-01,00:51,2022-01-01,00:34,2022-01-01,00:36,128.0


In [ ]:
import json

# Convertir DataFrame a formato JSON compatible con Next.js y guardarlo en un archivo
df.to_json('tripsData.json', orient="records", lines=True)

# Leer el contenido del archivo JSON línea por línea
with open('tripsData.json', 'r') as f:
    lines = f.readlines()

# Convertir cada línea a un diccionario
data = [json.loads(line) for line in lines]

# Ahora data es una lista de diccionarios
print(data)



In [1]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/ffvh_analytics/ffvh_analytics_part_0_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = dd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
df.head()

NameError: name 'dd' is not defined

In [ ]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("c3uy-2p5r", limit=16218)

# Convert to pandas DataFrame (16218, 12)
results_df = pd.DataFrame.from_records(results)

In [ ]:
# Seleccionar las variables relevantes
variables_seleccionadas = [
    'geo_place_name', 'geo_type_name', 'start_date',  # Variables de ubicación y tiempo
    'indicator_id', 'name', 'measure', 'measure_info',  # Variables relacionadas con el tráfico
    'data_value',  # Variable de valor de los datos
    'unique_id', 'geo_join_id'  # Variables de identificación
]

# Seleccionar las columnas relevantes del DataFrame
df_tratado = results_df[variables_seleccionadas].copy()

# Convertir las columnas de fecha y hora a tipos de datos datetime
df_tratado['start_date'] = pd.to_datetime(df_tratado['start_date'])

# Convertir la columna 'data_value' a tipo numérico si es posible
df_tratado['data_value'] = pd.to_numeric(df_tratado['data_value'], errors='coerce')

# Manejar los valores nulos o valores no numéricos en 'data_value'
# Por ejemplo, podrías optar por eliminar las filas con valores nulos en 'data_value'
df_tratado = df_tratado.dropna(subset=['data_value'])